In [1]:
import pandas as pd
import numpy as np

import torch
import torch.optim as optim
import torch.nn.functional as F

torch.manual_seed(1)

from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import accuracy_score

In [2]:
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')
submit= pd.read_csv('sample_submit.csv')

In [3]:
train

,label,1x1,1x2,1x3,1x4,1x5,1x6,1x7,1x8,1x9,...,28x19,28x20,28x21,28x22,28x23,28x24,28x25,28x26,28x27,28x28
0,5,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,3,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,3,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,6,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
59995,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
59996,3,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
59997,9,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
59998,4,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [4]:
train_x = train.drop(['label'], axis =1)
test_x = test
train_y = pd.DataFrame(train.label)

In [5]:
train_x_tensor = torch.FloatTensor(train_x.values) #60000,784
train_y_tensor = torch.LongTensor(train_y.values) #60000,1

test_x_tensor = torch.FloatTensor(test_x.values)

In [6]:
W = torch.zeros((784,10), requires_grad = True)
b = torch.zeros(1, requires_grad = True)

In [7]:
optimizer = optim.Adam([W,b], lr=0.01)
total_epoch= 5000
for epoch in range(1,total_epoch+1):
    hypothesis = F.softmax(train_x_tensor.matmul(W) + b, dim = 1)
    
    y_one_hot = torch.zeros(len(train.label), 10)
    y_one_hot.scatter_(1, train_y_tensor, 1)
    
    cost = (y_one_hot * -torch.log(F.softmax(hypothesis, dim=1))).sum(dim=1).mean()
    optimizer.zero_grad()
    cost.backward()
    optimizer.step()
    
    if epoch % 100 == 0:
        print('Epoch {:4d}/{} Cost: {:.6f}'.format(
            epoch, total_epoch, cost.item()
        ))

Epoch  100/2000 Cost: 1.560460
Epoch  200/2000 Cost: 1.548509
Epoch  300/2000 Cost: 1.541005
Epoch  400/2000 Cost: 1.535008
Epoch  500/2000 Cost: 1.532030
Epoch  600/2000 Cost: 1.528523
Epoch  700/2000 Cost: 1.523185
Epoch  800/2000 Cost: 1.521754
Epoch  900/2000 Cost: 1.517091
Epoch 1000/2000 Cost: 1.517397
Epoch 1100/2000 Cost: 1.516589
Epoch 1200/2000 Cost: 1.514111
Epoch 1300/2000 Cost: 1.515371
Epoch 1400/2000 Cost: 1.515329
Epoch 1500/2000 Cost: 1.513707
Epoch 1600/2000 Cost: 1.514243
Epoch 1700/2000 Cost: 1.511019
Epoch 1800/2000 Cost: 1.514872
Epoch 1900/2000 Cost: 1.510992
Epoch 2000/2000 Cost: 1.510095


In [8]:
train_hypothesis = F.softmax(train_x_tensor.matmul(W) + b, dim = 1)
predict_train = torch.argmax(train_hypothesis, dim = 1)
print(accuracy_score(train.label, predict_train))
predict_hypothesis = F.softmax(test_x_tensor.matmul(W) + b, dim = 1)
predict = torch.argmax(predict_hypothesis, dim = 1)

0.9510166666666666


In [9]:
submit.label = (predict.detach().numpy().astype('int'))

In [10]:
submit.to_csv('2021_09_30_02.csv', index=False) #5000 0.01 0.92930